In [1]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm import load_node, Code, Computer
from aiida.common.exceptions import NotExistent, MultipleObjectsError

In [2]:
## -------------------------------------------------------
## List computers
qb = QueryBuilder()
qb.append(Computer)

for e in qb.all():
    print(e)
## -------------------------------------------------------

[<Computer: daint-s746 (daint.cscs.ch), pk: 1>]
[<Computer: daint-s746-normal (daint.cscs.ch), pk: 2>]


In [3]:
## -------------------------------------------------------
## List  codes
qb = QueryBuilder()
qb.append(Code)

for e in qb.all():
    print(e)
## -------------------------------------------------------

[<Code: Remote code 'py_stm_sts_6a7e3' on daint-s746-normal, pk: 2266, uuid: 6c06da7b-3dcf-4fe8-bc07-b2b8a9cf4b26>]
[<Code: Remote code 'py_eval_morbs_6a7e3' on daint-s746-normal, pk: 2265, uuid: e31902cb-31ec-42fd-ad44-b50c7019c967>]
[<Code: Remote code 'stm_sts_6a7e3' on daint-s746-normal, pk: 2264, uuid: 912864d1-6c79-4d7f-ba9b-3e488c621b26>]
[<Code: Remote code 'eval_morbs_6a7e3' on daint-s746-normal, pk: 2263, uuid: 93b1394f-a6d3-418b-a0a1-41301b6fc1a6>]
[<Code: Remote code 'q2r-6.1' on daint-s746-normal, pk: 1084, uuid: 64e560da-4786-495c-8d17-dc3cc2bf0bd4>]
[<Code: Remote code 'matdyn-6.1' on daint-s746-normal, pk: 1083, uuid: 9aa49b21-3826-42ba-be10-916eabfc92ff>]
[<Code: Remote code 'projwfc-6.1' on daint-s746-normal, pk: 1082, uuid: f53c4538-4c35-4c42-be3d-253fa7769990>]
[<Code: Remote code 'ph-6.1' on daint-s746-normal, pk: 1081, uuid: 2fe4aff7-16fb-453e-9a8b-1ff33199ed19>]
[<Code: Remote code 'pp-6.1' on daint-s746-normal, pk: 1080, uuid: 6e44c37a-8f03-485b-bae4-e83d1c7fad5

In [4]:
# 1. Select computer

computer_name = "daint-s746-normal"
computer = Computer.get(computer_name)

# 2. Set remote path where to install the codes

code_remote_path = "/users/keimre/aiida-soft"


## Retrieve scripts to remote machine

In [28]:
commit_hash = ! git ls-remote "https://github.com/eimrek/atomistic_tools.git"
version_id = commit_hash[0][:5]

code_dir_name = "at_"+version_id

# ssh key needs to be set

! ssh {computer.hostname} "wget https://github.com/eimrek/atomistic_tools/archive/master.zip -O {code_remote_path}/{code_dir_name}.zip"
! ssh {computer.hostname} "cd {code_remote_path}&& unzip -o {code_dir_name}.zip && rm -rf {code_dir_name} && mv atomistic_tools-master {code_dir_name}" 

--2018-06-05 15:07:41--  https://github.com/eimrek/atomistic_tools/archive/master.zip
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/eimrek/atomistic_tools/zip/master [following]
--2018-06-05 15:07:42--  https://codeload.github.com/eimrek/atomistic_tools/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.253.121, 192.30.253.120
Connecting to codeload.github.com (codeload.github.com)|192.30.253.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7840482 (7.5M) [application/zip]
Saving to: ‘/users/keimre/aiida-soft/at_6a7e3.zip’

     0K .......... .......... .......... .......... ..........  0%  227K 34s
    50K .......... .......... .......... .......... ..........  1%  466K 25s
   100K .......... .......... .......... .......... ..........  1% 67.8M 16s
   150

  6650K .......... .......... .......... .......... .......... 87%  578K 0s
  6700K .......... .......... .......... .......... .......... 88%  158M 0s
  6750K .......... .......... .......... .......... .......... 88%  210M 0s
  6800K .......... .......... .......... .......... .......... 89%  529M 0s
  6850K .......... .......... .......... .......... .......... 90%  107M 0s
  6900K .......... .......... .......... .......... .......... 90%  109M 0s
  6950K .......... .......... .......... .......... .......... 91%  127M 0s
  7000K .......... .......... .......... .......... .......... 92% 96.6M 0s
  7050K .......... .......... .......... .......... .......... 92%  127M 0s
  7100K .......... .......... .......... .......... .......... 93%  104M 0s
  7150K .......... .......... .......... .......... .......... 94%  120M 0s
  7200K .......... .......... .......... .......... .......... 94%  161M 0s
  7250K .......... .......... .......... .......... .......... 95%  206M 0s
  7300K ....

## Set up the remote codes

In [29]:
# Python preferably with MPI support and ASE

prepend_text  = "### code prepend_text start ###\n"
prepend_text += "module load cray-python/3.6.1.1\n"
prepend_text += "export PYTHONPATH=$PYTHONPATH:\"/users/keimre/soft/ase\"\n"
prepend_text += "### code prepend_text end ###\n"

code_dir_path = code_remote_path+"/"+code_dir_name

In [38]:
# Eval mol. orbitals

code_label = "py_eval_morbs_"+version_id

code_full_name = code_label + "@" + computer_name

try:
    Code.get_from_string(code_full_name)
except (NotExistent, MultipleObjectsError):            
    code = Code(remote_computer_exec=(computer, code_dir_path + "/evaluate_morbs_in_box_mpi.py"))
    code.label = code_label
    code.description = "Python code to evaluate mol. orbitals on a grid"
    #code.set_input_plugin_name("cp2k")
    code.set_prepend_text(prepend_text)
    code.set_append_text("")
    code.store()
    ! verdi code show "{code_full_name}"
else:
    print("Code '{}' already existent".format(code_full_name))       

 * PK:             2265
 * UUID:           e31902cb-31ec-42fd-ad44-b50c7019c967
 * Label:          py_eval_morbs_6a7e3
 * Description:    Python code to evaluate mol. orbitals on a grid
 * Default plugin: None
 * Used by:        0 calculations
 * Type:           remote
 * Remote machine: daint-s746-normal
 * Remote absolute path: 
   /users/keimre/aiida-soft/at_6a7e3/evaluate_morbs_in_box_mpi.py
 * prepend text:
   ### code prepend_text start ###
   module load cray-python/3.6.1.1
   export PYTHONPATH=$PYTHONPATH:"/users/keimre/soft/ase"
   ### code prepend_text end ###
   
 * append text:
   # No append text.


In [39]:
# stm & sts from npz

code_label = "py_stm_sts_"+version_id

code_full_name = code_label + "@" + computer_name

try:
    Code.get_from_string(code_full_name)
except (NotExistent, MultipleObjectsError):            
    code = Code(remote_computer_exec=(computer, code_dir_path + "/stm_sts_extrap_from_npz.py"))
    code.label = code_label
    code.description = "Python code to calculate STM and STS from the output of py_eval_morbs"
    #code.set_input_plugin_name("cp2k")
    code.set_prepend_text(prepend_text)
    code.set_append_text("")
    code.store()
    ! verdi code show "{code_full_name}"
else:
    print("Code '{}' already existent".format(code_full_name))     

 * PK:             2266
 * UUID:           6c06da7b-3dcf-4fe8-bc07-b2b8a9cf4b26
 * Label:          py_stm_sts_6a7e3
 * Description:    Python code to calculate STM and STS from the output of eval_morbs
 * Default plugin: None
 * Used by:        0 calculations
 * Type:           remote
 * Remote machine: daint-s746-normal
 * Remote absolute path: 
   /users/keimre/aiida-soft/at_6a7e3/stm_sts_extrap_from_npz.py
 * prepend text:
   ### code prepend_text start ###
   module load cray-python/3.6.1.1
   export PYTHONPATH=$PYTHONPATH:"/users/keimre/soft/ase"
   ### code prepend_text end ###
   
 * append text:
   # No append text.


# Alternative: setup remote python

In [19]:
prepend_text  = "### code prepend_text start ###\n"
prepend_text += "module load cray-python/3.6.1.1\n"
prepend_text += "export PYTHONPATH=$PYTHONPATH:\"/users/keimre/soft/ase\"\n"
prepend_text += "### code prepend_text end ###\n"

exe_path = ! ssh {computer.hostname} "module load cray-python/3.6.1.1 && which python"

code_label = "python3"

code_full_name = code_label + "@" + computer_name

try:
    Code.get_from_string(code_full_name)
except (NotExistent, MultipleObjectsError):            
    code = Code(remote_computer_exec=(computer, exe_path[0]))
    code.label = code_label
    code.description = "Python 3 with custom libraries"
    code.set_prepend_text(prepend_text)
    code.set_append_text("")
    code.store()
    ! verdi code show "{code_full_name}"
else:
    print("Code '{}' already existent".format(code_full_name))     

 * PK:             2444
 * UUID:           8c7308b7-1cf8-4d53-b0d2-02ef70f345da
 * Label:          python3
 * Description:    Python 3 with custom libraries
 * Default plugin: None
 * Used by:        0 calculations
 * Type:           remote
 * Remote machine: daint-s746-normal
 * Remote absolute path: 
   /opt/python/3.6.1.1/bin/python
 * prepend text:
   ### code prepend_text start ###
   module load cray-python/3.6.1.1
   export PYTHONPATH=$PYTHONPATH:"/users/keimre/soft/ase"
   ### code prepend_text end ###
   
 * append text:
   # No append text.
